In [69]:
import sympy

CYCLE_TIME = 0.01
G_ERR = 1
H_ERR = 10

In [70]:
gx,gy,gz,hx,hy,hz = sympy.symbols('g_x g_y g_z h_x h_y h_z')
g = sympy.Matrix([gx,gy,gz])
h = sympy.Matrix([hx,hy,hz])
X = sympy.Matrix([gx,gy,gz,hx,hy,hz])
X

Matrix([
[g_x],
[g_y],
[g_z],
[h_x],
[h_y],
[h_z]])

In [71]:
wx,wy,wz = sympy.symbols('omega_x omega_y omega_z')
w = sympy.Matrix([wx,wy,wz])
ax,ay,az = sympy.symbols('a_x a_y a_z')
a = sympy.Matrix([ax,ay,az])
mx,my,mz = sympy.symbols('m_x m_y m_z')
m = sympy.Matrix([mx,my,mz])

m

Matrix([
[m_x],
[m_y],
[m_z]])

In [72]:
Q = sympy.eye(6)
for i in range(3):
    Q[i,i]     = G_ERR
    Q[i+3,i+3] = H_ERR
R = 0.1*sympy.eye(6)

In [73]:
P = 100*Q
P

Matrix([
[100,   0,   0,    0,    0,    0],
[  0, 100,   0,    0,    0,    0],
[  0,   0, 100,    0,    0,    0],
[  0,   0,   0, 1000,    0,    0],
[  0,   0,   0,    0, 1000,    0],
[  0,   0,   0,    0,    0, 1000]])

In [74]:
# dg = CYCLE_TIME*w.cross(g)
# dh = CYCLE_TIME*w.cross(h)

F = CYCLE_TIME*sympy.Matrix([
    [0, -wz, wy, 0, 0, 0],
    [wz, 0, -wx, 0, 0, 0],
    [-wy, wx, 0, 0, 0, 0],
    [0, 0, 0, 0, -wz, wy],
    [0, 0, 0, wz, 0, -wx],
    [0, 0, 0, -wy, wx, 0],
])

F

Matrix([
[            0, -0.01*omega_z,  0.01*omega_y,             0,             0,             0],
[ 0.01*omega_z,             0, -0.01*omega_x,             0,             0,             0],
[-0.01*omega_y,  0.01*omega_x,             0,             0,             0,             0],
[            0,             0,             0,             0, -0.01*omega_z,  0.01*omega_y],
[            0,             0,             0,  0.01*omega_z,             0, -0.01*omega_x],
[            0,             0,             0, -0.01*omega_y,  0.01*omega_x,             0]])

In [75]:
Z = sympy.Matrix([ax,ay,az,mx,my,mz])
Z

Matrix([
[a_x],
[a_y],
[a_z],
[m_x],
[m_y],
[m_z]])

In [76]:
# Predict
X_kp1_est = X + F*X
P_kp1_est = F*P*F.T * Q
X_kp1_est

Matrix([
[ g_x - 0.01*g_y*omega_z + 0.01*g_z*omega_y],
[ 0.01*g_x*omega_z + g_y - 0.01*g_z*omega_x],
[-0.01*g_x*omega_y + 0.01*g_y*omega_x + g_z],
[ h_x - 0.01*h_y*omega_z + 0.01*h_z*omega_y],
[ 0.01*h_x*omega_z + h_y - 0.01*h_z*omega_x],
[-0.01*h_x*omega_y + 0.01*h_y*omega_x + h_z]])

## Update
- measure residual: $ y_k = z - h(\hat{x}_{k|k-1}) $
- innovation covariance: $ S_k = H_k P_{k|k-1} H_k^T + R_k $
- Kalman gain: $ K_k = P_{k|k-1} H_k^T S_k^{-1} $
- Update state estimate: $ \hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k y_k $
- Update covariance estimate: $ P_{k|k} = (I - K_k H_k)P_{k|k-1} $

In [77]:
# Update
hx = X_kp1_est
Y = Z - hx
S = P_kp1_est+R
K = P_kp1_est*S.inv()
x_kp1 = X_kp1_est + K*Y
P_kp1 = (sympy.eye(6)- K)*P_kp1_est
x_kp1

Matrix([
[ g_x - 0.01*g_y*omega_z + 0.01*g_z*omega_y + (-0.01*omega_x*omega_y*(-1.0e-10*omega_x**4*omega_y**3*omega_z**3 - 1.0e-9*omega_x**4*omega_y**3*omega_z - 1.0e-10*omega_x**4*omega_y*omega_z**5 - 2.0e-9*omega_x**4*omega_y*omega_z**3 - 1.0e-8*omega_x**4*omega_y*omega_z - 2.0e-10*omega_x**2*omega_y**5*omega_z**3 - 2.0e-9*omega_x**2*omega_y**5*omega_z - 4.0e-10*omega_x**2*omega_y**3*omega_z**5 - 8.0e-9*omega_x**2*omega_y**3*omega_z**3 - 4.0e-8*omega_x**2*omega_y**3*omega_z - 2.0e-10*omega_x**2*omega_y*omega_z**7 - 6.0e-9*omega_x**2*omega_y*omega_z**5 - 6.0e-8*omega_x**2*omega_y*omega_z**3 - 2.0e-7*omega_x**2*omega_y*omega_z - 1.0e-10*omega_y**7*omega_z**3 - 1.0e-9*omega_y**7*omega_z - 3.0e-10*omega_y**5*omega_z**5 - 6.0e-9*omega_y**5*omega_z**3 - 3.0e-8*omega_y**5*omega_z - 3.0e-10*omega_y**3*omega_z**7 - 9.0e-9*omega_y**3*omega_z**5 - 9.0e-8*omega_y**3*omega_z**3 - 3.0e-7*omega_y**3*omega_z - 1.0e-10*omega_y*omega_z**9 - 4.0e-9*omega_y*omega_z**7 - 6.0e-8*omega_y*omega_z**5 - 4.0e-